In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.conv_learner import *
from fastai.dataset import *

from pathlib import Path
import json
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects

In [4]:
# Code taken from https://github.com/mutaku/xml2json

import xml.etree.cElementTree as ET
import simplejson, optparse, sys, os

def elem_to_internal(elem,strip=1):

    """Convert an Element into an internal dictionary (not JSON!)."""

    d = {}
    for key, value in elem.attrib.items():
        d['@'+key] = value

    # loop over subelements to merge them
    for subelem in elem:
        v = elem_to_internal(subelem,strip=strip)
        tag = subelem.tag
        value = v[tag]
        try:
            # add to existing list for this tag
            d[tag].append(value)
        except AttributeError:
            # turn existing entry into a list
            d[tag] = [d[tag], value]
        except KeyError:
            # add a new non-list entry
            d[tag] = value
    text = elem.text
    tail = elem.tail
    if strip:
        # ignore leading and trailing whitespace
        if text: text = text.strip()
        if tail: tail = tail.strip()

    if tail:
        d['#tail'] = tail

    if d:
        # use #text element if other attributes exist
        if text: d["#text"] = text
    else:
        # text is the value if no attributes
        d = text or None
    return {elem.tag: d}

def elem2json(elem, strip=1):

    """Convert an ElementTree or Element into a JSON string."""

    if hasattr(elem, 'getroot'):
        elem = elem.getroot()
    return simplejson.dumps(elem_to_internal(elem,strip=strip))

def xml2json(xmlstring,strip=1):

    """Convert an XML string into a JSON string."""

    elem = ET.fromstring(xmlstring)
    return elem2json(elem,strip=strip)

In [5]:
PATH = Path('/home/paperspace/fastai/nba_court_vision/data/annotations')

In [6]:
annotations = [json.loads(xml2json(open(f).read()))['annotation'] for f in PATH.iterdir()]

In [7]:
len(annotations)

131

In [8]:
PATH = Path('/home/paperspace/data/pascal')
trn_j = json.load((PATH / 'pascal_train2007.json').open())
IMAGES,ANNOTATIONS,CATEGORIES = ['images', 'annotations', 'categories']
FILE_NAME,ID,IMG_ID,CAT_ID,BBOX = 'file_name','id','image_id','category_id','bbox'

cats = dict((o[ID], o['name']) for o in trn_j[CATEGORIES])
trn_fns = dict((o[ID], o[FILE_NAME]) for o in trn_j[IMAGES])
trn_ids = [o[ID] for o in trn_j[IMAGES]]

JPEGS = 'VOCdevkit/VOC2007/JPEGImages'
IMG_PATH = PATH/JPEGS

In [12]:
PATH = Path('/home/paperspace/fastai/data/team')

In [13]:
output = []
for annot in annotations:
    a_obj = annot['object']
    bb_dicts = [a['bndbox'] for a in a_obj]
    o = a_obj[0]

    bs = [[int(a['xmin']), 
           int(a['ymin']), 
           int(a['xmax'])-int(a['xmin']), 
           int(a['ymax'])-int(a['ymin'])
          ] for a in bb_dicts]

    cats = [a['name'] for a in a_obj]
    area = [b[2]*b[3] for b in bs]
    fn = [str(PATH/annot['filename'])] * len(bs)

    for a, b, c, f in zip(area, bs, cats, fn):
        output.append({'area':a, 'bbox':b, 'category_id':c, 'id':f})

In [15]:
pickle.dump(output, open('annotations_list.pkl','wb'))

In [19]:
cat2idx = {'player':0, 'hoop':1, 'ft':2, 'ball':3, 'clogo':4}
idx2cat = {v:k for k,v in cat2idx.items()}

In [16]:
output

[{'area': 6840,
  'bbox': [819, 347, 57, 120],
  'category_id': 'player',
  'id': '/home/paperspace/fastai/data/team/bulls_bucks_h1_1.jpg'},
 {'area': 9577,
  'bbox': [852, 378, 61, 157],
  'category_id': 'player',
  'id': '/home/paperspace/fastai/data/team/bulls_bucks_h1_1.jpg'},
 {'area': 10922,
  'bbox': [1137, 438, 86, 127],
  'category_id': 'player',
  'id': '/home/paperspace/fastai/data/team/bulls_bucks_h1_1.jpg'},
 {'area': 20670,
  'bbox': [1130, 353, 130, 159],
  'category_id': 'player',
  'id': '/home/paperspace/fastai/data/team/bulls_bucks_h1_1.jpg'},
 {'area': 9179,
  'bbox': [1087, 345, 67, 137],
  'category_id': 'player',
  'id': '/home/paperspace/fastai/data/team/bulls_bucks_h1_1.jpg'},
 {'area': 12948,
  'bbox': [890, 253, 78, 166],
  'category_id': 'player',
  'id': '/home/paperspace/fastai/data/team/bulls_bucks_h1_1.jpg'},
 {'area': 13410,
  'bbox': [787, 218, 90, 149],
  'category_id': 'player',
  'id': '/home/paperspace/fastai/data/team/bulls_bucks_h1_1.jpg'},
 {'ar

In [26]:
from collections import defaultdict
def hw_bb(bb): return np.array([bb[1], bb[0], bb[3]+bb[1]-1, bb[2]+bb[0]-1])
def bb_hw(a): return np.array([a[1],a[0],a[3]-a[1]+1,a[2]-a[0]+1])

In [23]:
trn_anno = collections.defaultdict(list)

In [24]:
for o in output:
    bb = hw_bb(o['bbox'])
    trn_anno[o['id']].append((bb, cat2idx[o['category_id']]))

In [9]:
def draw_rect(ax, b):
    patch = ax.add_patch(patches.Rectangle(b[:2], *b[-2:], fill=False, edgecolor='white', lw=2))
    draw_outline(patch, 4)

In [10]:
def show_img(im, figsize=None, ax=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    return ax

In [11]:
def draw_outline(o, lw):
    o.set_path_effects([patheffects.Stroke(
        linewidth=lw, foreground='black'), patheffects.Normal()])

In [35]:
def get_lrg(b):
    if not b: raise Exception()
    b = sorted(b, key=lambda x: np.product(x[0][-2:] - x[0[:2]]), reverse=True)
    return b[0]

In [31]:
trn_lrg_ann = {a: get_lrg(b) for a,b in trn_anno.items()}

TypeError: 'int' object is not subscriptable